In [15]:

import inspect
import itertools
import logging

logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())

class State:
    def __init__(self, name: str, on_enter:(str or list) =None, on_exit:(str or list) =None,
                 ignore_invalid_triggers: bool =False):
        
        self.name = name
        self.on_enter = listify(on_enter) if on_enter else []
        self.on_exit = listify(on_exit) if on_exit else []
        self.ignore_invalid_triggers = ignore_invalid_triggers
    
    def __str__(self):
        return str(self.name) + "*/*" + str(self.ignore_invalid_triggers) + "*/*" + str(self.on_enter) + "*/*" +\
                str(self.on_exit)
    
    def enter(self, event_data: EventData):
        """ Triggered when a state is entered. """
        logger.debug("%sEntering state %s. Processing callbacks...", event_data.machine.id, self.name)
        for oe in self.on_enter:
            event_data.machine._callback(oe, event_data)
        logger.info("%sEntered state %s", event_data.machine.id, self.name)

    def exit(self, event_data: EventData):
        """ Triggered when a state is exited. """
        logger.debug("%sExiting state %s. Processing callbacks...", event_data.machine.id, self.name)
        for oe in self.on_exit:
            event_data.machine._callback(oe, event_data)
        logger.info("%sExited state %s", event_data.machine.id, self.name)

    def add_callback(self, trigger: str, func: str):
        """ Add a new enter or exit callback."""
        callback_list = getattr(self, 'on_' + trigger)
        callback_list.append(func)
    
if __name__ == '__main__':
    s = State("drink")
    print(s)


drink*/*False*/*[]*/*[]


In [5]:


class Condition:
    """
        This class should not be initialized or called from outside a
        Transition instance, and exists at module level.      
    """
    __slots__ = 'func', 'target'
    def __init__(self, func: str, target: bool=True):
            """
            :param func  : Name of the condition-checking callable
            :param target: Indicates the target state--i.e., when True,
                           the condition-checking callback should return True to pass,
                           and when False, the callback should return False to pass.
            """
            self.func = func
            self.target = target


    def check(self, event_data: EventData):
        """ Check whether the condition passes.
            :param event_data: An EventData instance to pass to the
                               condition (if event sending is enabled) or to extract arguments
                               from (if event sending is disabled). Also contains the data
                               model attached to the current machine which is used to invoke
                               the condition.
            :return: Method :)
        """
        predicate = getattr(event_data.model, self.func) if isinstance(self.func, string_types) else self.func

        if event_data.machine.send_event:
            return predicate(event_data) == self.target
        else:
            return predicate(*event_data.args, **event_data.kwargs) == self.target
        
        

In [2]:


class Transition:
    __slots__ = 'source', 'dest', 'conditions', 'unless', 'before', 'after', 'prepare'
    def __init__(self, source: str, dest: str, conditions:(str or list) =None,
                 unless:(str or list) =None, before:(str or list) =None,
                 after:(str or list) =None, prepare:(str or list)=None):
        """
        Args:
            :param source     : The name of the source State.
            :param dest       : The name of the destination State.
            :param conditions : Condition(s) that must pass in order for
                                the transition to take place. Either a string providing the
                                name of a callable, or a list of callables. For the transition
                                to occur, ALL callables must return True.
            :param unless     : Condition(s) that must return False in order
                for the transition to occur. Behaves just like conditions arg
                otherwise.
            :param before     : callbacks to trigger before the
                transition.
            :param after      : callbacks to trigger after the transition.
            :param prepare    : callbacks to trigger before conditions are checked
        """
        self.source = source
        self.dest = dest
        self.prepare = [] if prepare is None else listify(prepare)
        self.before = [] if before is None else listify(before)
        self.after = [] if after is None else listify(after)

        self.conditions = []
        if conditions is not None:
            for c in listify(conditions):
                self.conditions.append(Condition(c))
        if unless is not None:
            for u in listify(unless):
                self.conditions.append(Condition(u, target=False))

    def execute(self, event_data: EventData):
        """ Execute the transition.
            :param event: An instance of class EventData.
            :returns    : boolean indicating whether or not the transition was
                          successfully executed (True if successful, False if not).
        """
        logger.debug("%sInitiating transition from state %s to state %s...",
                     event_data.machine.id, self.source, self.dest)
        machine = event_data.machine

        for func in self.prepare:
            machine._callback(func, event_data)
            logger.debug("Executed callback '%s' before conditions." % func)

        for c in self.conditions:
            if not c.check(event_data):
                logger.debug("%sTransition condition failed: %s() does not " +
                             "return %s. Transition halted.", event_data.machine.id, c.func, c.target)
                return False
        for func in self.before:
            machine._callback(func, event_data)
            logger.debug("%sExecuted callback '%s' before transition.", event_data.machine.id, func)

        self._change_state(event_data)

        for func in self.after:
            machine._callback(func, event_data)
            logger.debug("%sExecuted callback '%s' after transition.", event_data.machine.id, func)
        return True

    def _change_state(self, event_data: EventData):
        event_data.machine.get_state(self.source).exit(event_data)
        event_data.machine.set_state(self.dest, event_data.model)
        event_data.update(event_data.model)
        event_data.machine.get_state(self.dest).enter(event_data)

    def add_callback(self, trigger: str, func: str):
        """ Add a new before, after, or prepare callback.
            :param trigger: The type of triggering event. Must be one of'before', 'after' or 'prepare'.
            :param func   : The name of the callback function.
        """
        callback_list = getattr(self, trigger)
        callback_list.append(func)


In [ ]:

class Machine:
    

In [ ]:


class Event:
    pass

In [3]:


class EventData:
   
    def __init__(self, state: State, event: Event, machine: Machine, model: object, args: list, kwargs: dict):
        """
            :param state   : The State from which the Event was triggered.
            :param event   : The triggering Event.
            :param machine : The current Machine instance.
            :param model   : The model/object the machine is bound to.
            :param args    : Optional positional arguments from trigger method
                             to store internally for possible later use.
            :param kwargs  : Optional keyword arguments from trigger method
                             to store internally for possible later use.
        """
        self.state = state
        self.event = event
        self.machine = machine
        self.model = model
        self.args = args
        self.kwargs = kwargs

    def update(self, model):
        """ Updates the current State to accurately reflect the Machine. """
        self.state = self.machine.get_state(model.state)

